In [1]:
import regex as re
from pathlib import Path
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/hauraytheo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/hauraytheo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
def get_raw_files_per_language(data_base: Path) -> dict[str, list[Path]]:
    res = {}
    for language_dir in data_base.iterdir():
        raw_dir = language_dir / "raw-documents"
        res[language_dir.name] = list(raw_dir.iterdir())
    return res

def get_stopwords_for_language(language: str, stopwords_path: Path) -> set[str]:
    if language.lower() in ("eng", "en", "english"):
        return set(stopwords.words('english'))
    if language.lower() in ("ru", "russian"):
        return set(stopwords.words('russian'))
    if language.lower() in ("pt", "portuguese"):
        return set(stopwords.words('portuguese'))
    if language.lower() in ("bg", "bulgarian"):
        sw = open(stopwords_path / "bulgarian.txt").readlines()
        sw = [w.strip() for w in sw]
        return set(sw)
    if language.lower() in ("hi", "hindi"):
        sw = open(stopwords_path / "hindi.txt").readlines()
        sw = [w.strip() for w in sw]
        return set(sw)

def process_file(filepath: Path, stop_words: set[str]) -> list[str]:
    url_pattern = re.compile(r'https?://\S+') ## remove urls- lots of articles have them

    content = "".join(open(filepath).readlines())
    content = url_pattern.sub("", content)
    content = content.lower()
    content = re.sub(r'[^\w\s]', "", content) # non-word and non-whitespace characters
    content = re.sub(r'\d', "", content) # remove digits
    content = re.sub(r"\s+", " ", content) # remove newlines, duplicate whitespaces, tabs, ...
    tokens = [ token for token in content.split(" ") if token not in stop_words ]

    return tokens

def add_tags(tokens: list[str]) -> list[tuple[str, str, str]]:
    pos_tags = nltk.pos_tag(tokens)
    lemmatizer = WordNetLemmatizer()
    result = [(token, tag, lemmatizer.lemmatize(token, get_wordnet_pos(tag))) for token, tag in pos_tags]

    return result

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN 

def save_to_conllu(filepath: Path, content: list[tuple[str, str, str]], language: str):
    with open(filepath, "w", encoding='utf-8') as f:
        f.write("# token\tPOS\tLEM\n")
        for i, (token, pos, lem) in enumerate(content):
            f.write(f"{i+1}\t{token}\t{pos}\t{lem}\n")
        f.write("\n")

In [3]:
base_path = Path.cwd().parent / "data"
filepaths = get_raw_files_per_language(base_path  / "training_data_16_October_release")

for language in filepaths.keys():
    stop_words = get_stopwords_for_language(language, base_path /  "stopwords")
    Path(base_path / "tmp" / language ).mkdir(parents=True, exist_ok=True)
    for filepath in filepaths[language]:
        output_file = base_path / "tmp" / language / filepath.name
        if output_file.exists():
            output_file.unlink() #delete if file exists
        tokens = process_file(filepath, stop_words)
        tagged_tokens = add_tags(tokens)
        save_to_conllu(output_file, tagged_tokens, language)